<a href="https://colab.research.google.com/github/vinuri-g/ML_Assignment2_IT19113982_IT19143200_IT19165394_IT19173200/blob/sanda-d/cholesterol_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Cholesterol Risk Level Using Machine Learning**

Cholesterol is a type of fat which moves through our body through blood. Cholesterol is made from our body as well as contained in the food we take in. Cholesterol is important to make layers inside blood vessels which act as a gatekeeper for things to move in and out of body cells. But there is a certain level of cholesterol that is needed for this. That is why it is important for one’s health to maintain the level of cholesterol needed without letting it go below or above the required range. Using attributes such as age, gender, height, weight, smoking habits and calorie level, we can, predict the cholesterol risk of a person. 

In [ ]:
# Import libraries, features and settings

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from operator import truediv
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

Data Source : https://www.kaggle.com/code/sulianova/eda-cardiovascular-data/data

In [ ]:
# Read dataset and Creating a dataframe

df = pd.read_csv("cardio_train.csv", sep=";")